<a href="https://colab.research.google.com/github/ftcelepcikay/CS221-project/blob/main/Code/Fake_Review_Detection_with_BERT_v5_keep_prob%3D0_55_BATCH_SIZE_%3D_32_epoch%3D8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Fake Review Detection with BERT on TF Hub

If you’ve been following Natural Language Processing over the past year, you’ve probably heard of BERT: Bidirectional Encoder Representations from Transformers. It’s a neural network architecture designed by Google researchers that’s totally transformed what’s state-of-the-art for NLP tasks, like text classification, translation, summarization, and question answering.

Now that BERT's been added to [TF Hub](https://www.tensorflow.org/hub) as a loadable module, it's easy(ish) to add into existing Tensorflow text pipelines. In an existing pipeline, BERT can replace text embedding layers like ELMO and GloVE. Alternatively, [finetuning](http://wiki.fast.ai/index.php/Fine_tuning) BERT can provide both an accuracy boost and faster training time in many cases.

Here, we'll train a model to predict whether an IMDB movie review is positive or negative using BERT in Tensorflow with tf hub. Some code was adapted from [this colab notebook](https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb). Let's get started!

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
# import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In addition to the standard libraries we imported above, we'll need to install BERT's python package.

In [ ]:
!pip install bert-tensorflow==1.0.1

     |████████████████████████████████| 67 kB 5.2 MB/s 


In [ ]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [ ]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'OUTPUT_DIR_NAME5'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: OUTPUT_DIR_NAME5 *****


#Data

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls /content/gdrive/MyDrive/CS221/Dataset/

testing_matrix.csv   training_matrix_ext.csv
training_matrix.csv  validating_matrix.csv


In [ ]:
train = pd.read_csv(r'/content/gdrive/MyDrive/CS221/Dataset/training_matrix.csv')
valid = pd.read_csv(r'/content/gdrive/MyDrive/CS221/Dataset/validating_matrix.csv')
test = pd.read_csv(r'/content/gdrive/MyDrive/CS221/Dataset/testing_matrix.csv')

In [ ]:
train.head

<bound method NDFrame.head of              date  user_id  ...  title_word_count upper_case_word_count
0      2011-08-11    25002  ...                 3                     0
1      2011-03-22     7756  ...                11                     6
2      2011-02-11   139209  ...                13                     4
3      2011-07-30     2400  ...                13                     2
4      2011-05-23    32189  ...                10                     4
...           ...      ...  ...               ...                   ...
11119  2011-12-31    49996  ...                 6                     2
11120  2011-12-31    93088  ...                 2                     1
11121  2011-12-31    89621  ...                 4                     2
11122  2011-12-31   117284  ...                 2                     1
11123  2011-12-31    46249  ...                29                     3

[11124 rows x 13 columns]>

In [ ]:
train.loc[train.label == 1, 'label'] = 0
train.loc[train.label == -1, 'label'] = 1

valid.loc[valid.label == 1, 'label'] = 0
valid.loc[valid.label == -1, 'label'] = 1

test.loc[test.label == 1, 'label'] = 0
test.loc[test.label == -1, 'label'] = 1

In [ ]:
import string
train['char_count'] = train['review'].apply(len)
train['word_count'] = train['review'].apply(lambda x: len(x.split()))
train['word_density'] = train['char_count'] / (train['word_count']+1)
train['punctuation_count'] = train['review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
train['title_word_count'] = train['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
train['upper_case_word_count'] = train['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

valid['char_count'] = valid['review'].apply(len)
valid['word_count'] = valid['review'].apply(lambda x: len(x.split()))
valid['word_density'] = valid['char_count'] / (valid['word_count']+1)
valid['punctuation_count'] = valid['review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
valid['title_word_count'] = valid['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
valid['upper_case_word_count'] = valid['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

test['char_count'] = test['review'].apply(len)
test['word_count'] = test['review'].apply(lambda x: len(x.split()))
test['word_density'] = test['char_count'] / (test['word_count']+1)
test['punctuation_count'] = test['review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
test['title_word_count'] = test['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
test['upper_case_word_count'] = test['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))




In [ ]:
# reviewer_centric features #
#train.user_id.value_counts()

train['date.1'] = pd.to_datetime(train['date.1'])
train['day_of_week'] = train['date.1'].dt.day_name()
#train.day_of_week.value_counts()[:10]]
dofw = train.day_of_week.unique()
for day in range(len(dofw)):
        label=train['label'][train.day_of_week == dofw[day]]
        fake=len(label[label==1])
        real=len(label[label==0])
        
        print("***********Day is ",dofw[day])
        print("fake is ",fake)
        print("real is ",real)
        print("tot is ",fake+real)
        print("Ratio is ",100*(fake/(fake+real)))
        print("len is ",len(label))

train['user_id_no_of_review'] = train.groupby('user_id')['user_id'].transform('size')
train['user_id_ave_rating'] = train.groupby('user_id')['rating'].transform('mean')
#train['user_id_std_rating'] = train.groupby('user_id')['rating'].transform('std')
train['user_id_ave_no_words'] = train.groupby('user_id')['word_count'].transform('mean')
train['user_id_max_review_a_day'] = train['user_id_no_of_review']
grouped = train.groupby('user_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    train.loc[train.user_id == name,'user_id_max_review_a_day'] = df2
    #group['user_id_max_review_a_day']=df2

#train[:][train.user_id==9236].sort_index(axis = 0)
    
valid['date.1'] = pd.to_datetime(valid['date.1'])
valid['day_of_week'] = valid['date.1'].dt.day_name()
#valid.day_of_week.value_counts()[:10]]
dofw = valid.day_of_week.unique()
for day in range(len(dofw)):
        label=valid['label'][valid.day_of_week == dofw[day]]
        fake=len(label[label==1])
        real=len(label[label==0])
        
        print("***********Day is ",dofw[day])
        print("fake is ",fake)
        print("real is ",real)
        print("tot is ",fake+real)
        print("Ratio is ",100*(fake/(fake+real)))
        print("len is ",len(label))

valid['user_id_no_of_review'] = valid.groupby('user_id')['user_id'].transform('size')
valid['user_id_ave_rating'] = valid.groupby('user_id')['rating'].transform('mean')
#valid['user_id_std_rating'] = valid.groupby('user_id')['rating'].transform('std')
valid['user_id_ave_no_words'] = valid.groupby('user_id')['word_count'].transform('mean')
valid['user_id_max_review_a_day'] = valid['user_id_no_of_review']
grouped = valid.groupby('user_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    valid.loc[valid.user_id == name,'user_id_max_review_a_day'] = df2
    #group['user_id_max_review_a_day']=df2

#valid[:][valid.user_id==9236].sort_index(axis = 0) 

test['date.1'] = pd.to_datetime(test['date.1'])
test['day_of_week'] = test['date.1'].dt.day_name()
#test.day_of_week.value_counts()[:10]]
dofw = test.day_of_week.unique()
for day in range(len(dofw)):
        label=test['label'][test.day_of_week == dofw[day]]
        fake=len(label[label==1])
        real=len(label[label==0])
        
        print("***********Day is ",dofw[day])
        print("fake is ",fake)
        print("real is ",real)
        print("tot is ",fake+real)
        print("Ratio is ",100*(fake/(fake+real)))
        print("len is ",len(label))

test['user_id_no_of_review'] = test.groupby('user_id')['user_id'].transform('size')
test['user_id_ave_rating'] = test.groupby('user_id')['rating'].transform('mean')
#test['user_id_std_rating'] = test.groupby('user_id')['rating'].transform('std')
test['user_id_ave_no_words'] = test.groupby('user_id')['word_count'].transform('mean')
test['user_id_max_review_a_day'] = test['user_id_no_of_review']
grouped = test.groupby('user_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    test.loc[test.user_id == name,'user_id_max_review_a_day'] = df2
    #group['user_id_max_review_a_day']=df2

#test[:][test.user_id==9236].sort_index(axis = 0) 

***********Day is  Thursday
fake is  764
real is  755
tot is  1519
Ratio is  50.29624753127058
len is  1519
***********Day is  Tuesday
fake is  800
real is  788
tot is  1588
Ratio is  50.377833753148614
len is  1588
***********Day is  Friday
fake is  758
real is  701
tot is  1459
Ratio is  51.953392734749826
len is  1459
***********Day is  Saturday
fake is  756
real is  744
tot is  1500
Ratio is  50.4
len is  1500
***********Day is  Monday
fake is  858
real is  913
tot is  1771
Ratio is  48.4472049689441
len is  1771
***********Day is  Sunday
fake is  790
real is  903
tot is  1693
Ratio is  46.66272888363851
len is  1693
***********Day is  Wednesday
fake is  836
real is  758
tot is  1594
Ratio is  52.446675031367626
len is  1594
***********Day is  Tuesday
fake is  150
real is  161
tot is  311
Ratio is  48.231511254019296
len is  311
***********Day is  Thursday
fake is  129
real is  128
tot is  257
Ratio is  50.19455252918288
len is  257
***********Day is  Saturday
fake is  137
real is 

In [ ]:
# product_centric features #
#train.prod_id.value_counts()

train['prod_id_no_of_review'] = train.groupby('prod_id')['prod_id'].transform('size') + 1000.
train['prod_id_ave_rating'] = train.groupby('prod_id')['rating'].transform('mean') + 1000.
#train['prod_id_std_rating'] = train.groupby('prod_id')['rating'].transform('std') + 1000.
train['prod_id_ave_no_words'] = train.groupby('prod_id')['word_count'].transform('mean') + 1000.
train['prod_id_max_review_a_day'] = train['prod_id_no_of_review'] + 1000.
grouped = train.groupby('prod_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    train.loc[train.prod_id == name,'prod_id_max_review_a_day'] = df2 + 1000.
    #group['prod_id_max_review_a_day']=df2

#train[:][train.prod_id==9236].sort_index(axis = 0)

valid['prod_id_no_of_review'] = valid.groupby('prod_id')['prod_id'].transform('size') + 1000.
valid['prod_id_ave_rating'] = valid.groupby('prod_id')['rating'].transform('mean') + 1000.
#valid['prod_id_std_rating'] = valid.groupby('prod_id')['rating'].transform('std') + 1000.
valid['prod_id_ave_no_words'] = valid.groupby('prod_id')['word_count'].transform('mean') + 1000.
valid['prod_id_max_review_a_day'] = valid['prod_id_no_of_review']
grouped = valid.groupby('prod_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    valid.loc[valid.prod_id == name,'prod_id_max_review_a_day'] = df2 + 1000.
    #group['prod_id_max_review_a_day']=df2

#valid[:][valid.prod_id==9236].sort_index(axis = 0)  

test['prod_id_no_of_review'] = test.groupby('prod_id')['prod_id'].transform('size') + 1000.
test['prod_id_ave_rating'] = test.groupby('prod_id')['rating'].transform('mean') + 1000.
#test['prod_id_std_rating'] = test.groupby('prod_id')['rating'].transform('std') + 1000.
test['prod_id_ave_no_words'] = test.groupby('prod_id')['word_count'].transform('mean') + 1000.
test['prod_id_max_review_a_day'] = test['prod_id_no_of_review']
grouped = test.groupby('prod_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    test.loc[test.prod_id == name,'prod_id_max_review_a_day'] = df2 + 1000.
    #group['prod_id_max_review_a_day']=df2

#test[:][test.prod_id==9236].sort_index(axis = 0)  

In [ ]:
train['total']=train.user_id.astype(str) + ' '+ train.prod_id.astype(str) + ' ' + train.rating.astype(str) + ' '+ \
train.user_id_no_of_review.astype(str) + ' ' + train.user_id_ave_rating.astype(str) + ' '+\
train.user_id_ave_no_words.astype(str) + ' ' + train.user_id_max_review_a_day.astype(str) + ' '+\
train.prod_id_no_of_review.astype(str) + ' ' + train.prod_id_ave_rating.astype(str) + ' '+\
train.prod_id_ave_no_words.astype(str) + ' ' + train.prod_id_max_review_a_day.astype(str) + ' '+\
train.day_of_week.astype(str) + ' '+train.review  

#train['total']=train.user_id.astype(str) + ' '+ train.prod_id.astype(str) + ' ' + train.rating.astype(str) + ' '+  train.review
#test['total']=test.user_id.astype(str) + ' '+ test.prod_id.astype(str) + ' ' + test.rating.astype(str) + ' '+test.review

valid['total']=valid.user_id.astype(str) + ' '+ valid.prod_id.astype(str) + ' ' + valid.rating.astype(str) + ' '+ \
valid.user_id_no_of_review.astype(str) + ' ' + valid.user_id_ave_rating.astype(str) + ' '+\
valid.user_id_ave_no_words.astype(str) + ' ' + valid.user_id_max_review_a_day.astype(str) + ' '+\
valid.prod_id_no_of_review.astype(str) + ' ' + valid.prod_id_ave_rating.astype(str) + ' '+\
valid.prod_id_ave_no_words.astype(str) + ' ' + valid.prod_id_max_review_a_day.astype(str) + ' '+\
valid.day_of_week.astype(str) + ' '+ valid.review  

test['total']=test.user_id.astype(str) + ' '+ test.prod_id.astype(str) + ' ' + test.rating.astype(str) + ' '+ \
test.user_id_no_of_review.astype(str) + ' ' + test.user_id_ave_rating.astype(str) + ' '+\
test.user_id_ave_no_words.astype(str) + ' ' + test.user_id_max_review_a_day.astype(str) + ' '+\
test.prod_id_no_of_review.astype(str) + ' ' + test.prod_id_ave_rating.astype(str) + ' '+\
test.prod_id_ave_no_words.astype(str) + ' ' + test.prod_id_max_review_a_day.astype(str) + ' '+\
test.day_of_week.astype(str) + ' '+ test.review  

First, let's download the dataset, hosted by Stanford. The code below, which downloads, extracts, and imports the IMDB Large Movie Review Dataset, is borrowed from [this Tensorflow tutorial](https://www.tensorflow.org/hub/tutorials/text_classification_with_tf_hub).

In [ ]:
test.total.head

<bound method NDFrame.head of 0       102359 514 5.0 1 5.0 7.0 1 1013.0 1003.4615384...
1       45961 763 5.0 1 5.0 76.0 1 1002.0 1005.0 1044....
2       78578 379 5.0 1 5.0 211.0 1 1021.0 1004.333333...
3       85624 419 5.0 1 5.0 19.0 1 1007.0 1003.7142857...
4       33686 101 4.0 1 4.0 14.0 1 1032.0 1004.375 107...
                              ...                        
9995    21818 841 3.0 1 3.0 513.0 1 1058.0 1003.396551...
9996    7790 706 4.0 1 4.0 300.0 1 1017.0 1003.8235294...
9997    101508 540 4.0 1 4.0 51.0 1 1009.0 1004.111111...
9998    8119 914 4.0 1 4.0 45.0 1 1015.0 1003.8 1116.3...
9999    3521 453 3.0 2 3.5 95.5 1 1025.0 1003.68 1157....
Name: total, Length: 10000, dtype: object>

To keep train fast, we'll take a sample of 5000 train and test examples, respectively.

In [ ]:
train.columns

Index(['date', 'user_id', 'prod_id', 'date.1', 'review', 'label', 'rating',
       'char_count', 'word_count', 'word_density', 'punctuation_count',
       'title_word_count', 'upper_case_word_count', 'day_of_week',
       'user_id_no_of_review', 'user_id_ave_rating', 'user_id_ave_no_words',
       'user_id_max_review_a_day', 'prod_id_no_of_review',
       'prod_id_ave_rating', 'prod_id_ave_no_words',
       'prod_id_max_review_a_day', 'total'],
      dtype='object')

In [ ]:
from sklearn.utils import shuffle
train = shuffle(train, random_state=0)
valid = shuffle(valid, random_state=0)
test = shuffle(test, random_state=0)

For us, our input data is the 'sentence' column and our label is the 'polarity' column (0, 1 for negative and positive, respecitvely)

In [ ]:
DATA_COLUMN = 'total'
LABEL_COLUMN = 'label'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [ ]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

valid_InputExamples = valid.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [ ]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Great--we just learned that the BERT model we're using expects lowercase data (that's what stored in tokenization_info["do_lower_case"]) and we also loaded BERT's vocab file. We also created a tokenizer, which breaks words into word pieces:

In [ ]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [ ]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
valid_features = bert.run_classifier.convert_examples_to_features(valid_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 11124


INFO:tensorflow:Writing example 0 of 11124


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


INFO:tensorflow:label: 1 (id = 1)


#Creating a model

Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [ ]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.55)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for train, evaluation, and prediction.

In [ ]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [ ]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 8.0
# Warmup is a period of time where the learning rate 
# is small and gradually increases--usually helps train.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 100
SAVE_SUMMARY_STEPS = 100

In [ ]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [ ]:
num_train_steps

2781

In [ ]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [ ]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': 'OUTPUT_DIR_NAME5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f94da91bc50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'OUTPUT_DIR_NAME5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f94da91bc50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [ ]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

In [ ]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

Instructions for updating:
Use standard file APIs to delete files with this prefix.


INFO:tensorflow:global_step/sec: 0.0259275


INFO:tensorflow:global_step/sec: 0.0259275


INFO:tensorflow:loss = 0.18896419, step = 501 (3856.907 sec)


INFO:tensorflow:loss = 0.18896419, step = 501 (3856.907 sec)


Now let's use our test data to see how well our model did:

In [ ]:
valid_input_fn = run_classifier.input_fn_builder(
    features=valid_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
estimator.evaluate(input_fn=valid_input_fn, steps=None)

In [ ]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

In [ ]:
predictions = estimator.predict(test_input_fn)
#list(predictions)[0]
pred = pd.DataFrame(list(predictions))
pred.head()

In [ ]:
import numpy as np
pred[['log_p0', 'log_p1']] = pd.DataFrame(pred.probabilities.tolist()) 
pred['p1']= np.exp(pred['log_p1'])

pred.head()

In [ ]:
test = (test).reset_index()

In [ ]:
(test.label == pred.labels).value_counts()

In [ ]:
pred['true_labels'] = test.label
pred['abs_dif'] = np.abs(pred.true_labels-pred.p1)
pred['review'] = test['review']
pred.head()

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(pred.true_labels , pred.p1)
auc = metrics.roc_auc_score(pred.true_labels , pred.p1)
print("Auc ROC is ",auc)
precision, recall, thresholds = metrics.precision_recall_curve(pred.true_labels , pred.p1)
auc = metrics.auc(recall, precision)
print("Auc PRC is ",auc)
f1 = metrics.f1_score(pred.true_labels, pred.labels)
print("F1 score is",f1)
print("Log loss is ",metrics.log_loss(pred.true_labels , pred.p1))

In [ ]:
import matplotlib.pyplot as plt
# plot the roc curve for the model
plt.plot(fpr, tpr, linestyle='--', label='BERT')
#plt.plot(fpr_ll, tpr_ll, linestyle='-', label='Logistic Regression')
#pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()

In [ ]:
results = pd.DataFrame()
results['prob'] = pred.p1
results['predictions'] = pred.labels
results['labels'] = pred.true_labels
results.to_csv('/content/gdrive/My Drive/Colab Notebooks/BERT.csv') 

In [ ]:
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
estimator.evaluate(input_fn=train_input_fn, steps=None)